# RNU4-2
[ReNU syndrome (RENU)](https://omim.org/entry/620851) is caused by heterozygous mutation in the RNU4-2 gene.

RNU4-2 is one of two U4 snRNA genes.

[Chen Y, et al. (2024)](https://pubmed.ncbi.nlm.nih.gov/38991538/) report:
> The n.64_65insT variant is one of six single base insertions that we observe in the 18 bp critical region in individuals with NDD, in a total of 100 individuals across cohorts. By contrast, single base insertions are very rare in population cohorts. Although we do also observe some SNVs in this region in individuals with NDD, our initial data suggest these SNVs may result in a milder phenotype. However, given this observation is based on only four fully phenotyped individuals, it needs to be confirmed in larger cohorts. 

In [1]:
import hpotk
import gpsea

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2024-08-13')
print(f'Loaded HPO v{hpo.version}')
print(f"Using gpsea version {gpsea.__version__}")

Loaded HPO v2024-08-13
Using gpsea version 0.9.2


# RNU4-2
We user the [Matched Annotation from NCBI and EMBL-EBI (MANE)](https://www.ncbi.nlm.nih.gov/refseq/MANE/) transcript for RNU4-2. RNU4-2 is non-coding and thus there is no protein identifier.

In [2]:
gene_symbol = 'RNU4-2'
mane_tx_id = 'NR_003137.3'

In [3]:
from ppktstore.registry import configure_phenopacket_registry
from gpsea.preprocessing import load_phenopackets
from gpsea.preprocessing import configure_caching_cohort_creator

phenopacket_store_release = '0.1.21'  # Update, if necessary
registry = configure_phenopacket_registry()

with registry.open_phenopacket_store(release=phenopacket_store_release) as ps:
    phenopackets = tuple(ps.iter_cohort_phenopackets(gene_symbol))

cohort_creator = configure_caching_cohort_creator(hpo)

cohort, qc_results = load_phenopackets(phenopackets, cohort_creator)  
qc_results.summarize()

Individuals Processed: 100%|██████████| 61/61 [00:16<00:00,  3.72 individuals/s]
Validated under permissive policy
No errors or warnings were found


In [4]:
from gpsea.view import CohortViewer
viewer = CohortViewer(hpo)
viewer.process(cohort=cohort, transcript_id=mane_tx_id)

n,HPO Term
49,Absent speech
48,Intellectual disability
46,Delayed ability to walk
45,Seizure
42,Hypotonia
38,Short stature
37,Failure to thrive
35,Severe global developmental delay
35,Constipation
32,Strabismus


# Genotype phenotype correlation analysis

In [5]:
from gpsea.analysis.pcats import configure_hpo_term_analysis
analysis = configure_hpo_term_analysis(hpo)

from gpsea.analysis.clf import prepare_classifiers_for_terms_of_interest
pheno_predicates = prepare_classifiers_for_terms_of_interest(
    cohort=cohort,
    hpo=hpo,
)

In [6]:
from gpsea.analysis.predicate import variant_key
from gpsea.analysis.clf import monoallelic_classifier


is_n64_65insT = variant_key("12_120291839_120291839_T_TA") # n.64_65insT
is_b = variant_key("12_120291835_120291835_G_GT") #12_120291835_120291835_G_GT
is_c = variant_key("12_120291839_120291839_T_TC") # 12_120291839_120291839_T_TC
is_d = variant_key("12_120291826_120291826_T_TA") # 12_120291826_120291826_T_TA

is_insertion = is_n64_65insT | is_b | is_c | is_d 


is_n64_65insT_predicate = monoallelic_classifier(
    a_predicate=is_n64_65insT,
    b_predicate=~is_n64_65insT,
    a_label= "n.64_65insT", 
    b_label="other"
)

is_insertion_predicate = monoallelic_classifier(
    a_predicate=is_insertion,
    b_predicate=~is_insertion,
    a_label="insertion", 
    b_label="other"
)

In [7]:
n64_65insT_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_clf=is_n64_65insT_predicate,
    pheno_clfs=pheno_predicates,
)
from gpsea.view import MtcStatsViewer

viewer = MtcStatsViewer()
viewer.process(n64_65insT_result)

Code,Reason,Count
HMF01,Skipping term with maximum frequency that was less than threshold 0.4,145
HMF03,Skipping term because of a child term with the same individual counts,10
HMF08,Skipping general term,92
HMF09,Skipping term with maximum annotation frequency that was less than threshold 0.4,234


In [8]:
from gpsea.view import summarize_hpo_analysis

summarize_hpo_analysis(hpo=hpo, result=n64_65insT_result)

Allele group                                       n.64_65insT         other  \
                                                         Count Percent Count   
Severe global developmental delay [HP:0011344]           33/38     87%   2/7   
Moderate global developmental delay [HP:0011343]          5/38     13%   5/7   
Absent speech [HP:0001344]                               45/51     88%   4/8   
Hypotonia [HP:0001252]                                   46/49     94%   5/8   
Hearing impairment [HP:0000365]                           6/38     16%   3/7   
Primary microcephaly [HP:0011451]                        19/34     56%   1/5   
Seizure [HP:0001250]                                     37/52     71%   8/8   
Constipation [HP:0002019]                                32/48     67%   3/7   
Delayed ability to walk [HP:0031936]                     40/41     98%   6/7   
Strabismus [HP:0000486]                                  29/49     59%   3/7   
Tented philtrum [HP:0011825]                             19/44     43%   2/8   
Intrauterine growth retardation [HP:0001511]             16/39     41%   1/5   
Secondary microcephaly [HP:0005484]                      15/43     35%   3/6   
Failure to thrive [HP:0001508]                           33/47     70%   4/7   
Growth delay [HP:0001510]                                38/38    100%   6/6   
Delayed gross motor development [HP:0002194]             40/41     98%   6/6   
Motor delay [HP:0001270]                                 50/50    100%   7/7   
Neurodevelopmental delay [HP:0012758]                    53/53    100%   8/8   
Neurodevelopmental abnormality [HP:0012759]              53/53    100%   8/8   
Feeding difficulties [HP:0011968]                        33/35     94%   5/5   
Abdominal symptom [HP:0011458]                           40/40    100%   5/5   
Short stature [HP:0004322]                               33/43     77%   5/7   
Global developmental delay [HP:0001263]                  53/53    100%   8/8   
Intellectual disability [HP:0001249]                     46/46    100%   7/7   
Aplasia/Hypoplasia involving the central nervou...       43/43    100%   6/6   
Morphological central nervous system abnormalit...       48/48    100%   6/6   
Nystagmus [HP:0000639]                                   22/44     50%   3/7   
Abnormal involuntary eye movements [HP:0012547]          22/22    100%   3/3   
Decreased body weight [HP:0004325]                       33/33    100%   4/4   
Microcephaly [HP:0000252]                                41/41    100%   6/6   
Abnormal axial skeleton morphology [HP:0009121]          46/46    100%   7/7   
Aplasia/Hypoplasia of the cerebrum [HP:0007364]          41/41    100%   6/6   
Abnormal cerebral morphology [HP:0002060]                44/44    100%   6/6   
Abnormal forebrain morphology [HP:0100547]               44/44    100%   6/6   
Abnormal brain morphology [HP:0012443]                   45/45    100%   6/6   
Abnormal speech pattern [HP:0002167]                     52/52    100%   6/6   
Abnormal communication [HP:0034434]                      52/52    100%   6/6   
Delayed speech and language development [HP:000...       52/52    100%   6/6   
Abnormality of the mouth [HP:0000153]                    36/36    100%   3/3   
Abnormal oral cavity morphology [HP:0000163]             36/36    100%   3/3   
Abnormal eyelid morphology [HP:0000492]                  29/29    100%   3/3   
Abnormal ocular adnexa morphology [HP:0030669]           29/29    100%   3/3   
Abnormal muscle tone [HP:0003808]                        47/47    100%   5/5   
Abnormal lip morphology [HP:0000159]                     29/29    100%   3/3   
Abnormal nasal morphology [HP:0005105]                   26/26    100%   2/2   
Abnormal conjugate eye movement [HP:0000549]             29/29    100%   3/3   
Gastroesophageal reflux [HP:0002020]                     23/48     48%   3/7   
Abnormal esophagus physiology [HP:0025270]               23/23    100%   3/3   


In [9]:
insertion_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_clf=is_insertion_predicate,
    pheno_clfs=pheno_predicates,
)
summarize_hpo_analysis(hpo=hpo, result=insertion_result)

Allele group                                       insertion         other  \
                                                       Count Percent Count   
Moderate global developmental delay [HP:0011343]        6/41     15%   4/4   
Severe global developmental delay [HP:0011344]         35/41     85%   0/4   
Absent speech [HP:0001344]                             47/54     87%   2/5   
High forehead [HP:0000348]                              3/56      5%   2/5   
Diabetes insipidus [HP:0000873]                         9/48     19%   3/5   
Hypotonia [HP:0001252]                                 48/52     92%   3/5   
Failure to thrive [HP:0001508]                         36/50     72%   1/4   
Constipation [HP:0002019]                              34/51     67%   1/4   
Delayed ability to walk [HP:0031936]                   43/44     98%   3/4   
Short stature [HP:0004322]                             36/46     78%   2/4   
Seizure [HP:0001250]                                   40/55     73%   5/5   
Primary microcephaly [HP:0011451]                      19/36     53%   1/3   
Nystagmus [HP:0000639]                                 24/47     51%   1/4   
Secondary microcephaly [HP:0005484]                    16/45     36%   2/4   
Growth delay [HP:0001510]                              41/41    100%   3/3   
Delayed gross motor development [HP:0002194]           43/44     98%   3/3   
Motor delay [HP:0001270]                               53/53    100%   4/4   
Neurodevelopmental delay [HP:0012758]                  56/56    100%   5/5   
Neurodevelopmental abnormality [HP:0012759]            56/56    100%   5/5   
Feeding difficulties [HP:0011968]                      36/38     95%   2/2   
Abdominal symptom [HP:0011458]                         43/43    100%   2/2   
Global developmental delay [HP:0001263]                56/56    100%   5/5   
Intellectual disability [HP:0001249]                   49/49    100%   4/4   
Aplasia/Hypoplasia involving the central nervou...     45/45    100%   4/4   
Morphological central nervous system abnormalit...     50/50    100%   4/4   
Abnormal involuntary eye movements [HP:0012547]        24/24    100%   1/1   
Decreased body weight [HP:0004325]                     36/36    100%   1/1   
Microcephaly [HP:0000252]                              43/43    100%   4/4   
Abnormal axial skeleton morphology [HP:0009121]        49/49    100%   4/4   
Aplasia/Hypoplasia of the cerebrum [HP:0007364]        43/43    100%   4/4   
Abnormal cerebral morphology [HP:0002060]              46/46    100%   4/4   
Abnormal forebrain morphology [HP:0100547]             46/46    100%   4/4   
Abnormal brain morphology [HP:0012443]                 47/47    100%   4/4   
Abnormal speech pattern [HP:0002167]                   54/54    100%   4/4   
Abnormal communication [HP:0034434]                    54/54    100%   4/4   
Delayed speech and language development [HP:000...     54/54    100%   4/4   
Abnormality of the mouth [HP:0000153]                  37/37    100%   2/2   
Abnormal oral cavity morphology [HP:0000163]           37/37    100%   2/2   
Abnormal eyelid morphology [HP:0000492]                30/30    100%   2/2   
Abnormal ocular adnexa morphology [HP:0030669]         30/30    100%   2/2   
Abnormal muscle tone [HP:0003808]                      49/49    100%   3/3   
Abnormal lip morphology [HP:0000159]                   30/30    100%   2/2   
Abnormal nasal morphology [HP:0005105]                 27/27    100%   1/1   
Strabismus [HP:0000486]                                30/52     58%   2/4   
Abnormal conjugate eye movement [HP:0000549]           30/30    100%   2/2   
Gastroesophageal reflux [HP:0002020]                   24/51     47%   2/4   
Abnormal esophagus physiology [HP:0025270]             24/24    100%   2/2   
Tented philtrum [HP:0011825]                           19/47     40%   2/5   
Abnormality of the philtrum [HP:0000288]               23/23    100%   2/2   
Abnormal upper lip mor

In [10]:
from gpsea.analysis.clf import sex_classifier
mf_result = analysis.compare_genotype_vs_phenotypes(
    cohort=cohort,
    gt_clf=sex_classifier(),
    pheno_clfs=pheno_predicates,
)

summarize_hpo_analysis(hpo, mf_result)

Sex                                                FEMALE           MALE  \
                                                    Count Percent  Count   
Reduced circulating growth hormone concentratio...   0/11      0%  11/22   
Feeding difficulties [HP:0011968]                   16/18     89%  22/22   
Gastroesophageal reflux [HP:0002020]                 9/24     38%  17/31   
Delayed gross motor development [HP:0002194]        19/20     95%  27/27   
Short stature [HP:0004322]                          18/22     82%  20/28   
Intrauterine growth retardation [HP:0001511]         8/17     47%   9/27   
Hypotonia [HP:0001252]                              24/26     92%  27/31   
Facial hypotonia [HP:0000297]                        4/10     40%   5/16   
Severe global developmental delay [HP:0011344]      15/20     75%  20/25   
Absent speech [HP:0001344]                          23/27     85%  26/32   
Primary microcephaly [HP:0011451]                    9/16     56%  11/23   
Failure to thrive [HP:0001508]                      15/23     65%  22/31   
Nystagmus [HP:0000639]                              10/22     45%  15/29   
Strabismus [HP:0000486]                             14/23     61%  18/33   
Growth delay [HP:0001510]                           21/21    100%  23/23   
Delayed ability to walk [HP:0031936]                19/20     95%  27/28   
Motor delay [HP:0001270]                            25/25    100%  32/32   
Neurodevelopmental delay [HP:0012758]               28/28    100%  33/33   
Neurodevelopmental abnormality [HP:0012759]         28/28    100%  33/33   
Abdominal symptom [HP:0011458]                      20/20    100%  25/25   
Global developmental delay [HP:0001263]             28/28    100%  33/33   
Intellectual disability [HP:0001249]                23/23    100%  30/30   
Aplasia/Hypoplasia involving the central nervou...  21/21    100%  28/28   
Morphological central nervous system abnormalit...  25/25    100%  29/29   
Abnormal involuntary eye movements [HP:0012547]     10/10    100%  15/15   
Decreased body weight [HP:0004325]                  15/15    100%  22/22   
Constipation [HP:0002019]                           15/23     65%  20/32   
Seizure [HP:0001250]                                20/27     74%  25/33   
Microcephaly [HP:0000252]                           20/20    100%  27/27   
Abnormal axial skeleton morphology [HP:0009121]     24/24    100%  29/29   
Aplasia/Hypoplasia of the cerebrum [HP:0007364]     20/20    100%  27/27   
Abnormal cerebral morphology [HP:0002060]           22/22    100%  28/28   
Abnormal forebrain morphology [HP:0100547]          22/22    100%  28/28   
Abnormal brain morphology [HP:0012443]              23/23    100%  28/28   
Abnormal speech pattern [HP:0002167]                27/27    100%  31/31   
Abnormal communication [HP:0034434]                 27/27    100%  31/31   
Delayed speech and language development [HP:000...  27/27    100%  31/31   
Abnormality of the mouth [HP:0000153]               19/19    100%  20/20   
Abnormal oral cavity morphology [HP:0000163]        19/19    100%  20/20   
Abnormal eyelid morphology [HP:0000492]             14/14    100%  18/18   
Abnormal ocular adnexa morphology [HP:0030669]      14/14    100%  18/18   
Abnormal muscle tone [HP:0003808]                   25/25    100%  27/27   
Abnormal lip morphology [HP:0000159]                16/16    100%  16/16   
Abnormal nasal morphology [HP:0005105]              10/10    100%  18/18   
Abnormal conjugate eye movement [HP:0000549]        14/14    100%  18/18   
Abnormal esophagus physiology [HP:0025270]            9/9    100%  17/17   
Tented philtrum [HP:0011825]                        11/26     42%  10/26   
Abnormality of the philtrum [HP:0000288]            14/14    100%  11/11   
Abnormal upper lip morphology [HP:0000177]          14/14    100%  11/11   
Recurrent maladaptive behavior [HP:5200241]         14/14    100%  12/12   
Atypical behavior [HP:0000708]                  

# Summary

In [11]:
from gpseacs.report import GpseaAnalysisReport, GPAnalysisResultSummary


fet_results = (
    GPAnalysisResultSummary.from_multi(
        result=n64_65insT_result,
    ),
    GPAnalysisResultSummary.from_multi(
        result=insertion_result,
    ),
    GPAnalysisResultSummary.from_multi(
        result=mf_result
    )
)

caption = "No previous statistical analysis of correlations with PTPN11 missense variants identified in the medical literature."
report = GpseaAnalysisReport(name=gene_symbol, 
                             cohort=cohort, 
                             fet_results=fet_results,
                             gene_symbol=gene_symbol,
                             mane_tx_id=mane_tx_id,
                             mane_protein_id="",
                             caption=caption)

In [12]:
from gpseacs.report import GpseaNotebookSummarizer
summarizer = GpseaNotebookSummarizer(hpo=hpo, gpsea_version=gpsea.__version__)
summarizer.summarize_report(report=report)

In [13]:
summarizer.process_latex(report=report)

Output to ../../supplement/tex/RNU4-2_summary_draft.tex
